In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/visualization-project-group/surprise-map-assignment/master/hate_crime.csv",usecols=['STATE_NAME', 'DATA_YEAR', 'BIAS_DESC', 'OFFENDER_RACE'])

df2 = df.groupby(['STATE_NAME','DATA_YEAR']).agg('count')

df2['Anti-Asian'] = df.query("BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['BIAS_DESC']
df2['Anti-Black or African American'] = df.query("BIAS_DESC == 'Anti-Black or African American'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-White'] = df.query("BIAS_DESC == 'Anti-White'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-American Indian or Alaska Native'] = df.query("BIAS_DESC == 'Anti-American Indian or Alaska Native'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-Native Hawaiian or Other Pacific Islander'] = df.query("BIAS_DESC == 'Anti-Native Hawaiian or Other Pacific Islander'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']

df2['Anti-Asian Offender Race Black'] = df.query("OFFENDER_RACE == 'Black or African American' & BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-Asian Offender Race White'] = df.query("OFFENDER_RACE == 'White' & BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-Asian Offender Race American Indian or Alaska Native'] = df.query("OFFENDER_RACE == 'American Indian or Alaska Native' & BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-Asian Offender Race Asian'] = df.query("OFFENDER_RACE == 'Asian' & BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']
df2['Anti-Asian Offender Race Native Hawaiian or Other Pacific Islander'] = df.query("OFFENDER_RACE == 'Native Hawaiian or Other Pacific Islander' & BIAS_DESC == 'Anti-Asian'").groupby(['STATE_NAME','DATA_YEAR']).agg('count')['OFFENDER_RACE']

df2 = df2.fillna(0)
df2 = df2.drop(columns=['OFFENDER_RACE'])
df2 = df2.rename(columns={"BIAS_DESC": "Total Crime"})
df2 = df2.reset_index()

df2 = pd.melt(df2, id_vars = ["STATE_NAME", "DATA_YEAR"],
            var_name = "Indicator",
            value_name="Value")

df2




,STATE_NAME,DATA_YEAR,Indicator,Value
0,Alabama,1992,Total Crime,4.0
1,Alabama,1993,Total Crime,5.0
2,Alabama,2002,Total Crime,2.0
3,Alabama,2003,Total Crime,1.0
4,Alabama,2004,Total Crime,3.0
...,...,...,...,...
15274,Wyoming,2013,Anti-Asian Offender Race Native Hawaiian or Ot...,0.0
15275,Wyoming,2015,Anti-Asian Offender Race Native Hawaiian or Ot...,0.0
15276,Wyoming,2016,Anti-Asian Offender Race Native Hawaiian or Ot...,0.0
15277,Wyoming,2017,Anti-Asian Offender Race Native Hawaiian or Ot...,0.0


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash()

available_indicators = df2['Indicator'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='TOTAL_COUNT'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='TOTAL_COUNT)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Alabama'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df2['DATA_YEAR'].min(),
        max=df2['DATA_YEAR'].max(),
        value=df2['DATA_YEAR'].max(),
        step=None,
        marks={str(year): str(year) for year in df['DATA_YEAR'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df2[df2['DATA_YEAR'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator'] == yaxis_column_name]['STATE_NAME'],
            customdata=dff[dff['Indicator'] == yaxis_column_name]['STATE_NAME'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

def create_time_series(dff, axis_type, title):
    return {
        'data': [go.Scatter(
            x=dff['DATA_YEAR'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])

def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df2[df2['STATE_NAME'] == country_name]
    dff = dff[dff['Indicator'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df2[df2['STATE_NAME'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
